This demo aims to provide the basic codes for loading experimental sessions in which the neural data is recorded by Cerebus and the EMGs are recorded by DSPW wireless system.

**1. Data files**

The data files are put in a folder on FSM server

In [1]:
import os
base_path = 'Z:/limblab/lab_folder/Projects/wireless_EMG/Pop_demo/'
file_list = os.listdir(base_path)
print(file_list)

['Pop_20210902_PG_baseline_001.mat', 'Pop_20210902_PG_baseline_001.nev', 'Pop_20210902_PG_baseline_001.ns3', 'Pop_20210902_PG_baseline_001.pkl', 'Pop_20210902_PG_baseline_001.rhd']


* The *.nev* and *.ns3* files are recorded by the Cerebus system. The *.nev* file stores cortical signals, while the *.ns3* stores forces, video sync pulses etc.
* The *.rhd* files stores the EMGs collected by the DSPW wireless system. *.rhd* is also the data format created by INTAN and used by their products.
* The *.mat* file is generated by [xds_matlab](https://github.com/limblab/xds/tree/master/xds_matlab), containing all necessary information in the *.nev* and *.ns3* files.

**Notice** Sometimes if there's no *.nsx* file accoompanying the *.nev* file the xds_matlab script would throw an error when trying to bin the data. When binning the data xds calls [this function](https://github.com/limblab/ClassyDataAnalysis/blob/master/%40commonDataStructure/sanitizeTimeWindows.m) in [cds](https://github.com/limblab/ClassyDataAnalysis). The *cds* tries to find the proper aligned time window for all data types based on the start and end time for certain types of data in the *.nsx* file (could be EMGs, forces or kinematics). So if it fails to find these types of data in *.nsx* file it will assume the time window is [0, inf], therefore leading to errors.

**2. Download the xds Python codes for wireless EMG and include it in your Python path**

Use **git bash** to download the codes and switch to the right branch

    cd F:
   
    git clone https://github.com/limblab/xds
   
    git checkout wireless_EMG

Use the codes below to include xds in your path. For example, here the xds-Python path should be `F:\\xds\\xds_python`

In [ ]:
import sys
sys.path.append('F:\\xds\\xds_python')

**3. Load the .mat file and the .rhd file and save the data in .pkl for future use**

In [3]:
from xds import lab_data_DSPW_EMG
import fnmatch
import numpy as np

mat_list = np.sort(fnmatch.filter(os.listdir(base_path+'/'), "*.mat"))
rhd_list = np.sort(fnmatch.filter(os.listdir(base_path+'/'), "*.rhd"))
print(mat_list)
print(rhd_list)

bad_EMG = [16, 24, 25, 26] 
# Known bad EMG channels. Channels 24, 25, 26 have been proved to be mistakenly shorted on the adaptor board
# Channel 16 could be related to an electrode failure
# When putting the channel numbers here, the processing codes will automatically remove them. If 2 channels in here belong to a
# pair of EMG, the codes will remove the pair, otherwise the remaining EMG will be treated as a single-ended channel.

for each in zip(mat_list, rhd_list):
    data = lab_data_DSPW_EMG(base_path, each[0], each[1], bad_EMG, comb_filter = 0, art_reject = 1)
    data.save_to_pickle(base_path) 

['Pop_20210902_PG_baseline_001.mat']
['Pop_20210902_PG_baseline_001.rhd']

Reading Intan Technologies RHD2000 Data File, Version 1.5

n signal groups 7
Found 32 amplifier channels.
Found 0 auxiliary input channels.
Found 0 supply voltage channels.
Found 0 board ADC channels.
Found 2 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 906.298 seconds of data.  Amplifiers were sampled at 2.01 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
Done!  Elapsed time: 5.7 seconds
The numbers of these bad channels are []
For noisy channel 1DI, use only one single end channel.
For noisy channel BI, use only one single end channel.
For noisy channel TRI, use only one single end channel.
For noisy channel EDC1, use only one single end channel.
Rejecting high amplitude EMG artifacts.
Applying no

Now both the neural data and wireless EMGs were taken care of and saved in the *.pkl*. The single-ended EMGs were paired and differentiated. The bad channels were removed. High amplitude artifacts were rejected. The good channels were filtered to create envelops.

You can load the *.pkl* file to do more work at any time.

**4. Load the .pkl file**

In [2]:
import pickle

file_name = 'Pop_20210902_PG_baseline_001.pkl'

with open ( base_path + file_name, 'rb' ) as fp:
    dataset = pickle.load(fp)
print('The original bin size is %f s'%(dataset.bin_width))
    
EMG_names = dataset.EMG_names
    
bin_size = 0.025 # Change the bin size here
smooth_size = 0.05 # Change the smooth window size here

dataset.update_bin_data(bin_size)  
dataset.smooth_binned_spikes(bin_size, 'gaussian', smooth_size)

# Getting the data in trials
trial_info = dataset.get_trial_info('R')
spike = dataset.get_trials_data_spike_counts('R', 'gocue_time', 0.5, 'gocue_time', 1.5)
EMG = dataset.get_trials_data_EMG('R', 'gocue_time', 0.5, 'gocue_time', 1.5)

The original bin size is 0.001000 s
The new bin size is 0.0250 s


In [7]:
print(EMG_names)

['APB', 'FPB', 'Lum', 'PT', 'FDS1', 'FCR2', 'FCU1', 'FDP2', '1DI', '4DI', 'EPM', 'ECR', 'ECU', 'EDC1', 'TRI', 'BI']


In [8]:
print('There are %d trials in this session.'%(len(trial_info)))

There are 98 trials in this session.


In [11]:
print('Here are trial information for the 10th trial:')
print(trial_info[10])

Here are trial information for the 10th trial:
{'trial_result': 'R', 'trial_target_dir': 90.0, 'trial_gocue_time': 90.0853, 'trial_start_time': 89.31, 'trial_end_time': 91.659, 'trial_target_corners': array([-12.5 ,  16.75,  12.5 ,  13.25]), 'trial_target_center_x': 0.0, 'trial_target_center_y': 15.0}


In [13]:
print('In this session the y position of the targets are:')
print(set(dataset.trial_target_center_y))

In this session the y position of the targets are:
{11.0, 13.0, 15.0}


In [3]:
print('Here is the shape of the data within one trial')
print('Spikes:')
print(spike[0].shape)
print('EMGs:')
print(EMG[0].shape)

Here is the shape of the data within one trial
Spikes:
(80, 96)
EMGs:
(80, 16)
